## 증권리포트 크롤링

- 매일경제증권센터 증권리포트 항목 크롤링
- 검색어, 검색할 maxPage 설정

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd

#증권리포트 제목, 내용, url을 분리, 저장하는 함수
def get_report_inf(url):
    #headers = {'Accept-Encoding': 'deflate'}
    #res = requests.get(url=url, headers=headers)
    res = requests.get(url=url)
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')#, from_encoding="euc-kr"
    title = soup.find('p', {'class':'tit'}).text
    text = soup.find('div', {'id':'article_main'}).text
    report_data = {}
    report_data['title'] = title.replace("\r"," ").replace("\n"," ").replace("\t"," ").strip()  # 기사 제목
    report_data['text'] = text.replace("\n", " ").replace("\r", " ").replace("<br>", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
    report_data['url'] = url  # 기사 링크
    
    return report_data

In [2]:
#증권리포트 크롤링 data를 반환하는 함수

def get_report_list(page=1  # 뉴스 리스트 페이지
                      ):
    
    
    url='https://www.thebell.co.kr/free/content/article.asp?page={page}&svccode=00'
    
    res = requests.get(url.format(page=page))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')
    #print(soup)
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출    
    table = soup.find('div', {'class':'listBox'})
    alinks = table.findAll('a')
    
    report_list = []
    
    for alink in alinks:
        alink['href'] = 'https://www.thebell.co.kr/free/content/' + alink['href']
        report_url = alink['href']
        report_list.append(get_report_inf(report_url))
    
    return report_list

In [3]:
#증권리포트 크롤링 실행
articles = []

for num in range(1, 41):
    articles.extend(get_report_list(page=num))

print("Total articles: {:,}".format(len(articles)))
#print("="*20, "Article sample", "="*20)
#print(articles[0])

Total articles: 400


In [4]:
df = pd.DataFrame(articles)
df[['title']].head(10)

title
0  예금보험공사, '납골당' 영각사 채권회수 잰걸음           하이글로벌그룹과 ...
1  산업은행·수출입은행, 동아탱커 구조조정 '변수'           선박금융 제공, ...
2  신세계인터, 이마트 부진에 동반 몸살           국내패션 25% 의존 '데이...
3  하나은행 중국법인, 성장 정체…美·中 분쟁 외풍 탓           [은행 해외법...
4         '불매운동' GA업계의 자승자박           [thebell note]
5  신한은행, 유럽서 고전…컴플라이언스 비용 확대 탓           [은행 해외법인...
6  [유업 리포트]서울우유, 2조 시대 포부 달성할까⑤아시아 최대 규모 '양주 신공장'...
7  [유업 리포트]'1위 흔들' 서울우유, 발목잡는 협동조합 지배구조④낙농 조합원 이익...
8  키움증권, '아쉬운' 신라젠 CB 리스크관리           [thebell note]
9  크래프트, 비주류 '프로그램 투자·AI' 접목 차별화           [VC가 주...

## 기업리스트로 단어 Filtering
- 상장기업이름이 포함된 기사들로만 wordlist를 재구성한다

In [5]:
import mysql.connector 
from sqlalchemy import create_engine
db_host = '10.250.114.79'
db_name = 'mysql'
db_id = 'root'
db_pw = 'skcc'
cnx_str = 'mysql+mysqlconnector://' + db_id + ':' + db_pw + '@' + db_host + '/' + db_name + ''
engine = create_engine(cnx_str,echo=False)
with engine.connect() as con:
    rs = con.execute("SELECT * FROM co_list ")
    data = rs.fetchall()

print("DATA: ", data[0][1])

co_list = [] #기업리스트
#print(data)
for co_no,co_name in data:
    co_name = co_name.replace("\r", "")
    co_list.append(co_name)
    
#print(co_list)

DATA:  그린플러스


## Text preprocessing

- 형태소 분류작업 (정규화, 어근화작업도 수행)

In [6]:
from konlpy.tag import Okt
from collections import Counter
from nltk.tag import pos_tag
sentence = "Emma refused to permit us to obtain the refuse permit"
tagged_list = pos_tag(word_tokenize(sentence))
tagged_list

# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)


#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set([])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#stopwords.add(searchword)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
    #words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    words = pos_tag(word_tokenize(article['text']))
    tmp_words = []
    
    
    #for w, t in words:
    #    if t in tag_set and len(w) > 1 and w not in stopwords:
    #        tmp_words.append(w)
    #print(tmp_words)
    co_words = [w for w in tmp_words if w in co_list]  #기업리스트로 필터링작업
    #print(co_words)
    if len(co_words) > 0:  #기업리스트 단어들 따로 저장
        vocab.update(tmp_words)
        words_list.append((tmp_words, article))
    

print(len(words_list))
vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    # 기사 내 서로 다른 단어 개수가 5개 이상인 기사만 사용    
    if len(words) > 5:
        new_words_list.append((words, article))
        
print(len(new_words_list))

C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


283
283


In [7]:
print(new_words_list[0][0])

['기사', '더벨', '유료', '페이지', '기사', '부산', '해운', '동아', '탱커', '가전', '통한', '모색', '나선', '가운데', '나용', '계약', 'BBCHP', '대출', '약정', '기한', '이익', '상실', 'EOD', '선언', 'KDB', '산업', '은행', '수출입은행', '향후', '행보', '관심', '채권', '동아', '탱커', '관련', '비토', '보유', '만큼', 'BBCHP', '계약', '연장', '동의', '여부', '쟁점사항', '투자', '은행', 'IB', '업계', '동아', '탱커', '지난', '회생', '법원', '가전', '허가', '신청서', '제출', '채무자', '회사', '인수', '의향', '내비', '자비스', '자산운용', '오퍼스', 'PE', 'NH', '투자', '증권', '컨소시엄', '상해', '채권', 'BBCHP', '연장', '협상', '돌입', '동아', '탱커', '회생', '채무', 'BBCHP', '계약', '의거', '선박', '금융', '동아', '탱커', '운용', '선박', '이중', 'BBCHP', '계약', '관련', 'BBCHP', '계약', '조세피난처', '특수', '목적', '법인', 'SPC', '설립', '다시', '용선', '구조', '동아', '탱커', '국내', '금융기관', '선박', '금융', '활용', '선박', '대여', '판매', '사업', '영위', '부동산', '일종', '투자', '문제', '동아', '탱커', '해운업', '잘못', '분석', '선박', '가격', '발생', '대규모', '손실', '침체', '물동량', '선박', '가격', '장부', '결과', '채권', 'EOD', '선언', '대체', '선사', '거나', '선박', '작업', '착수', '이유', '이자지', '중단', '대출', '자금', '회수', '나선', '선박', '영영', '재기', '판단', '동아', '탱커', '금융기

## Build term-document matrix

In [8]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print('기사 갯수 : ' ,len(words_list), ' ', len(new_words_list))
print('vocab 갯수 : ' ,len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
#for i, (words, article) in enumerate(words_list_filtering):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

기사 갯수 :  283   283
vocab 갯수 :  7595
Term document matrix shape: (283, 7595)
  (0, 7541)	0.020609354558998223
  (0, 7525)	0.023231261136744518
  (0, 7524)	0.22060002182931115
  (0, 7519)	0.013858191664345296
  (0, 7505)	0.03640681000777743
  (0, 7394)	0.025861588068729752
  (0, 7391)	0.05037376247131239
  (0, 7388)	0.07654859986977527
  (0, 7343)	0.027012072764738906
  (0, 7341)	0.032636941971060915
  (0, 7326)	0.0266074434351652
  (0, 7318)	0.04216177191491788
  (0, 7317)	0.034384608518933525
  (0, 7315)	0.016892766486726444
  (0, 7314)	0.04216177191491788
  (0, 6990)	0.007079079067107317
  (0, 6939)	0.019375095017728702
  (0, 6936)	0.03714566107072689
  (0, 6870)	0.024570919411530424
  (0, 6837)	0.02826366584740628
  (0, 6819)	0.044210458109858686
  (0, 6755)	0.47149742674491707
  (0, 6723)	0.04216177191491788
  (0, 6713)	0.039291452228743085
  (0, 6588)	0.02400429151292163
  :	:
  (282, 1634)	0.012034384625640647
  (282, 1602)	0.014156314315220032
  (282, 1588)	0.026603978343933805
 

## Topic modeling

In [9]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 8  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (283, 8)
H shape: (8, 7595)


In [10]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:15]:
        print(vocab[index], end=' ')        
    print("\n", "="*50)

0th topic
펀드 운용 자산운용 투자 출자 설정 PE 해외 기업 대표 케이 그로 프랭클린 국내 파트너 
1th topic
항공 아시아나 그룹 인수 입찰 산업 미래에셋 HDC 대우 개발 참여 예비 KCGI 컨소시엄 회장 
2th topic
모채 발행 SK 회사채 조달 금리 신용등급 건설 예측 입금 자금 신용 시장 에너지 규모 
3th topic
바이오 브릿지 생산 CMO 기술 개발 SK 바이 넥스 의약품 CDMO 제약 직스 스톤 마일 
4th topic
상반기 영업 매출 부문 사업 수익 실적 이익 올해 중국 은행 대출 감소 지난해 증가 
5th topic
LG 디스플레이 로봇 전자 아바텍 협력 아바 투자 주가 베니 장비 기술 식각 보스 모델 
6th topic
상장 IPO 공모 기업 심사 거래소 녹십자 베어링 CS 증권 청약 시장 특례 코스닥 리츠 
7th topic
에스엠 지배구조 개편 라이 크기 지주사 계열 논의 자산운용 회사 기관 김앤장 방안 분율 분할 


In [11]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:5]:
        print(words_list[index][1]['title'])        
    print("\n", "="*80)

0th topic
기업구조혁신펀드 순항, 출자재원 소진 임박           블라인드 6개·프로젝트 펀드 4개 성사
삼성액티브-프랭클린 합병 무산, '플랜B' 있나           해외 운용사 제휴 가능성…자체 역량 강화는 '의문'
아이파트너스, 펀드설정 500억 돌파…운용보수 확대           [인사이드 헤지펀드]설립 2년 신생운용사…연말 순익 흑자전환 예상
키움PE, 1호 블라인드펀드 소진 '숨고르기'           1년만에 40% 투자 집행…관리 집중
그로쓰힐운용, 핵심인력 대거 영입           [인사이드 헤지펀드]메자닌 키맨 김홍범 상무·NH PI 이재호 이사, 그로쓰힐 '새 둥지'

1th topic
[아시아나항공 M&A]애경 vs 현대산업-미래대우 vs KCGI '3파전'오후 2시 마감…기존 원매자 세곳 응찰
[아시아나항공 M&A]예비입찰 함구령…GS그룹 참여했나 안했나정보차단 총력…입찰 이후에도 참여 가능성 열려있어
[아시아나항공 M&A]항공업 미래 불투명한데…인수전 뛰어드는 이유는기존 사업과 시너지·다각화 기대, 왕성한 현금창출력도 매력
[아시아나항공 M&A]SK·한화, 결국 불참…그래도 여전한 잠재후보군흥행실패에 베팅, 예비입찰 불참해도 본입찰 참여 가능
[아시아나항공 M&A]HDC현대산업개발, 유력 인수 후보로 깜짝 등장현금유동성 풍부…예비입찰 참여 여부 촉각

2th topic
SK㈜, 공모채 발행 추진…만기구조 고심           2500억 안팎…올해 세번째 조달, 연타석 오버부킹 기대
SK E&S, 회사채 발길 뚝…단기자금 활용           CP잔량 3000억…신용도 회복 안간힘
SK E&S, 회사채 발길 뚝…단기자금 활용           CP잔량 3000억…신용도 회복 안간힘
건설업 채권발행 활발…발행사·투자자 '윈윈'           [Market Watch]신용등급 'AA~BBB0'까지 회사채 발행…저금리 조달효과 만끽
SK에너지, 공모채 추진…15년물 도전할까           3000억 규모 타진

## Visualization

In [12]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


W shape: (283, 8)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 283 samples in 0.000s...
[t-SNE] Computed neighbors for 283 samples in 0.000s...
[t-SNE] Computed conditional probabilities for sample 283 / 283
[t-SNE] Mean sigma: 0.048670
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.491356
[t-SNE] KL divergence after 850 iterations: 0.351650


In [13]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [14]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [15]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [16]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [17]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [18]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 20, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
show(prepared_data)

(8, 7595)
7595


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8894/    [Ctrl-C to exit]


127.0.0.1 - - [05/Sep/2019 14:41:59] "GET / HTTP/1.1" 200 -



stopping Server...


## Simple search engine

In [47]:
query = 'pe'
query_index = word2id[query]  # 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
#print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
#print('랭크', ranks);



for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


KeyError: 'pe'